In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from torch.quantization import quantize_dynamic
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Utility functions
def print_size_of_model(model, name="Model"):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print(f'Size of {name} (MB): {os.path.getsize("temp.p") / 1e6}')
    os.remove('temp.p')

def accuracy(output, target):
    with torch.no_grad():
        batch_size = target.size(0)
        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        return correct[:1].view(-1).float().sum(0, keepdim=True).mul_(100.0 / batch_size).item()

# Define the AlexNet model
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(96),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(384),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(384),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),

            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),

            nn.Linear(4096, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train(model, trainloader, test_loader=None, num_epochs=10, device='cpu'):

    model.to(device)
    model.train()  # Set the model to training mode
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Print training epoch statistics
        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

        # Evaluate on test data after each epoch (if test_loader is provided)
        if test_loader is not None:
            test_accuracy = test(model, test_loader, cuda=(device == 'cuda'))
            print(f"Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2f}%\n")

    print("Training complete.")

def test(model, dataloader, cuda=False):

    correct = 0
    total = 0
    model.eval()  # Set the model to evaluation mode
    device = 'cuda' if cuda else 'cpu'
    model.to(device)

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy : {accuracy:.2f}%\n")
    return accuracy

In [ ]:
# Train the AlexNet model
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
model = AlexNet()
train(model, train_loader, num_epochs=10, device='cuda')
test(model, testloader, cuda=True)


Epoch [1/10], Loss: 1206.0651, Training Accuracy: 47.72%
Epoch [2/10], Loss: 828.8183, Training Accuracy: 64.53%
Epoch [3/10], Loss: 636.0511, Training Accuracy: 72.88%
Epoch [4/10], Loss: 644.7297, Training Accuracy: 72.61%
Epoch [5/10], Loss: 614.1888, Training Accuracy: 74.15%
Epoch [6/10], Loss: 667.9389, Training Accuracy: 72.05%
Epoch [7/10], Loss: 600.1991, Training Accuracy: 74.51%
Epoch [8/10], Loss: 576.4717, Training Accuracy: 75.83%
Epoch [9/10], Loss: 619.0132, Training Accuracy: 74.10%
Epoch [10/10], Loss: 635.8859, Training Accuracy: 73.65%
Training complete.
Test Accuracy : 74.04%



74.04

In [ ]:
def quantize_and_test(model, test_loader, quant_type):

    model.cpu()  # Quantization only supports CPU
    if quant_type == 'int8':
        quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
    elif quant_type == 'int16':
        quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.float16)
    elif quant_type == 'int4':
        print("Simulating INT4 using INT8 quantization.")
        quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
        for name, param in quantized_model.named_parameters():
            param.data = torch.round(param.data * 15) / 15  # Simulate 4-bit quantization
    else:
        raise ValueError(f"Unsupported quantization type: {quant_type}")

    # Test the quantized model
    quantized_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cpu(), labels.cpu()  # Ensure CPU tensors
            outputs = quantized_model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"{quant_type.upper()} Quantized Model Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
# Quantize and test the model
quant_types = ['int8', 'int16']
quantized_accuracies = {}
for qt in quant_types:
    quantized_accuracies[qt] = quantize_and_test(model, testloader, qt)

# Print results
print("\nQuantization Results:")
for qt, acc in quantized_accuracies.items():
    print(f"{qt.upper()} Accuracy: {acc:.2f}%")

INT8 Quantized Model Accuracy: 65.91%
INT16 Quantized Model Accuracy: 74.03%

Quantization Results:
INT8 Accuracy: 65.91%
INT16 Accuracy: 74.03%
